In [26]:
import numpy as np
from sklearn.metrics import classification_report
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Sequential

import cv2 as cv

In [46]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  'data/dataset/',
  label_mode='categorical',
  shuffle=True,
  seed=42,
  image_size=(32, 32),
)

test_ds = tf.keras.utils.image_dataset_from_directory(
  'data/test_dataset/',
  label_mode='categorical',
  shuffle=True,
  seed=42,
  image_size=(32, 32),
)

train_X, train_y = zip(*train_ds)
train_X = np.concatenate(train_X)
train_y = np.concatenate(train_y)

test_X, test_y = zip(*test_ds)
test_X = np.concatenate(test_X)
test_y = np.concatenate(test_y)


Found 10019 files belonging to 10 classes.
Found 353 files belonging to 10 classes.


In [61]:
def cvt_to_gray(images):
    gray_images = []
    for rgb_image in images:
        gray_image = cv.cvtColor(rgb_image, cv.COLOR_RGB2GRAY)
        gray_images.append(gray_image)
    gray_images = np.array(gray_images)
    gray_images = gray_images[..., np.newaxis]
    return gray_images

In [62]:
gray_train_X = cvt_to_gray(train_X)
gray_test_X = cvt_to_gray(test_X)

In [34]:
gray_train_X.shape

(10019, 32, 32, 1)

In [24]:
train_X.shape, train_y.shape

((10019, 32, 32, 3), (10019, 10))

In [3]:
x, y = next(iter(train_ds))

In [64]:
model = Sequential([
    layers.Input(shape=(None, None, 1)),
    layers.Resizing(32, 32),
    layers.Rescaling(1./255),
    layers.RandomFlip('horizontal'),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax'),
])

In [65]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_7 (Resizing)       (None, 32, 32, 1)         0         
                                                                 
 rescaling_7 (Rescaling)     (None, 32, 32, 1)         0         
                                                                 
 random_flip_7 (RandomFlip)  (None, 32, 32, 1)         0         
                                                                 
 conv2d_21 (Conv2D)          (None, 30, 30, 32)        320       
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 13, 13, 64)        18496     
                                                      

In [66]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [67]:
gray_train_X.shape, train_y.shape

((10019, 32, 32, 1), (10019, 10))

In [68]:
history = model.fit(gray_train_X, train_y, validation_data=(gray_test_X, test_y), epochs=40, batch_size=64)

Epoch 1/40
157/157 [==============================] - 5s 29ms/step - loss: 2.0897 - accuracy: 0.2033 - val_loss: 2.6293 - val_accuracy: 0.1955
Epoch 2/40
157/157 [==============================] - 5s 30ms/step - loss: 1.6198 - accuracy: 0.4111 - val_loss: 2.3192 - val_accuracy: 0.3201
Epoch 3/40
157/157 [==============================] - 5s 30ms/step - loss: 1.1978 - accuracy: 0.5778 - val_loss: 2.2954 - val_accuracy: 0.2946
Epoch 4/40
157/157 [==============================] - 5s 30ms/step - loss: 0.9686 - accuracy: 0.6588 - val_loss: 2.7615 - val_accuracy: 0.2493
Epoch 5/40
157/157 [==============================] - 5s 30ms/step - loss: 0.7757 - accuracy: 0.7345 - val_loss: 2.5831 - val_accuracy: 0.3258
Epoch 6/40
157/157 [==============================] - 5s 30ms/step - loss: 0.6434 - accuracy: 0.7797 - val_loss: 2.7856 - val_accuracy: 0.3541
Epoch 7/40
157/157 [==============================] - 5s 30ms/step - loss: 0.5154 - accuracy: 0.8260 - val_loss: 3.4623 - val_accuracy: 0.3484

In [22]:
X, y = zip(*test_ds)
X = np.concatenate(X)
y = np.concatenate(y)

In [203]:
X.shape, y.shape

((72, 32, 32, 3), (72, 10))

In [23]:
X, y = zip(*test_ds)
X = np.concatenate(X)
y = np.concatenate(y)

predictions = model.predict(X)
report = classification_report(np.argmax(y, axis=1), np.argmax(predictions, axis=1))
print(report)

12/12 [==============================] - 0s 26ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1
           3       0.55      0.66      0.60        50
           4       0.84      0.54      0.66        50
           5       0.35      0.74      0.47        50
           6       0.30      0.14      0.19        50
           7       1.00      0.04      0.08        50
           8       0.36      0.10      0.16        50
           9       0.36      0.60      0.45        50

    accuracy                           0.40       353
   macro avg       0.38      0.28      0.26       353
weighted avg       0.53      0.40      0.37       353

